## F値上げ作戦

#### 概要
- cepstrumsにある001,005,010,011,013の５つのデータを使用
- １つは評価用データとして他はtraindataとする。
- 評価用データを使ってF値を出す。
- ブーストする。
- 繰り返し、評価値を上げる。

#### 方針
- 学習にはLSTM,SVM等を使う。
- データを4:1に分けたあと4つ~~のtraindataを3:1に分ける~~
- ~~ブースト方法はxgboost,lightGBM~~
- active learning
 - ~~最初に3でtrainをし1のデータでtest~~
 - 4つでtrain
 - 評価用の１日のデータでtest
 - testの出力結果からあいまいな結果のデータをリスト化
 - traindataに前述のリストを追加
 - 再学習
 - 曖昧な結果が一通りなくなったら評価用データの別日のデータでtest
- activelearningの有効性を示す。

In [1]:
import os
from os import path
from argparse import ArgumentParser

import glob
import numpy as np
from numpy.lib.stride_tricks import as_strided
import pandas as pd

import tensorflow as tf
from keras import backend as K
from keras.utils import np_utils

import create_model
from create_model import read_csv,read_datas,test2main
import pandas as pd

Using TensorFlow backend.
/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:176: SyntaxWarning: name 'predicted' is assigned to before global declaration
  global predicted #モジュールのグローバル化


#### 組み合わせ

In [2]:
import itertools
directory_list = ["001", "005", "010", "011", "013"]
combi_list = list(itertools.combinations(directory_list,4))

In [3]:
for combi in combi_list:
    print(combi)
    test_data = [i for i in directory_list if not i in combi]
    print(test_data)
    test2main("/home/aj1m0n/jupyter_notebook/Datamining/20180731/cepstrums/",
              "./models/201808017/model/model%s_%s_%s_%s.json" %combi,
              "./models/201808017/eval/eval%s_%s_%s_%s.csv" %combi,
              "./models/201808017/weight/weight%s_%s_%s_%s.h5" %combi,
              "./models/201808017/predict/",
              100,
              40,
              180,
              3,
              1,
              list(combi),
              test_data
              )
    

('001', '005', '010', '011')
['013']


/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0
_________________________________________________________________
Trai

### predicted dataの処理

In [4]:
labelcsv = pd.read_csv("./cepstrums/013/013_20170512_000.label.csv")

In [2]:
create_model._label

AttributeError: module 'create_model' has no attribute '_label'

In [5]:
labelcsv["timestamp"]

0        2017-05-12T08:18:19.364
1        2017-05-12T08:18:20.364
2        2017-05-12T08:18:21.364
3        2017-05-12T08:18:22.364
4        2017-05-12T08:18:23.364
5        2017-05-12T08:18:24.363
6        2017-05-12T08:18:25.364
7        2017-05-12T08:18:26.364
8        2017-05-12T08:18:27.364
9        2017-05-12T08:18:28.364
10       2017-05-12T08:18:29.364
11       2017-05-12T08:18:30.364
12       2017-05-12T08:18:31.364
13       2017-05-12T08:18:32.364
14       2017-05-12T08:18:33.364
15       2017-05-12T08:18:34.364
16       2017-05-12T08:18:35.364
17       2017-05-12T08:18:36.364
18       2017-05-12T08:18:37.364
19       2017-05-12T08:18:38.364
20       2017-05-12T08:18:39.363
21       2017-05-12T08:18:40.363
22       2017-05-12T08:18:41.363
23       2017-05-12T08:18:42.363
24       2017-05-12T08:18:43.363
25       2017-05-12T08:18:44.363
26       2017-05-12T08:18:45.363
27       2017-05-12T08:18:46.363
28       2017-05-12T08:18:47.363
29       2017-05-12T08:18:48.363
          

In [6]:
pd.DataFrame(create_model.predicted).to_csv("./models/20180802/predicted013.csv",header=("NO_LABEL", "ROLL", "RUN", "DOOR"), index=False)

In [2]:
predicted013 = pd.read_csv("./models/20180802/predicted013.csv")
predicted013.head()

,NO_LABEL,ROLL,RUN,DOOR
0,0.167289,0.000181,0.832329,0.000201
1,0.261441,0.000992,0.737331,0.000236
2,0.337246,0.001861,0.660809,0.000085
3,0.381247,0.004716,0.613814,0.000223
4,0.362594,0.003354,0.633884,0.000168


In [5]:
predicted013["timestamp"] = labelcsv["timestamp"]

In [6]:
predicted013 = predicted013.set_index("timestamp")

In [5]:
predicted013.max

<bound method DataFrame.max of        NO_LABEL      ROLL       RUN          DOOR                timestamp
0      0.167289  0.000181  0.832329  2.011868e-04  2017-05-12T08:18:19.364
1      0.261441  0.000992  0.737331  2.357286e-04  2017-05-12T08:18:20.364
2      0.337246  0.001861  0.660809  8.459464e-05  2017-05-12T08:18:21.364
3      0.381247  0.004716  0.613814  2.231172e-04  2017-05-12T08:18:22.364
4      0.362594  0.003354  0.633884  1.679605e-04  2017-05-12T08:18:23.364
5      0.388502  0.003409  0.608009  7.991047e-05  2017-05-12T08:18:24.363
6      0.396020  0.006538  0.597119  3.229514e-04  2017-05-12T08:18:25.364
7      0.385846  0.005252  0.608648  2.540962e-04  2017-05-12T08:18:26.364
8      0.352676  0.002025  0.645235  6.428149e-05  2017-05-12T08:18:27.364
9      0.387179  0.005459  0.607093  2.687322e-04  2017-05-12T08:18:28.364
10     0.351297  0.005092  0.642937  6.743468e-04  2017-05-12T08:18:29.364
11     0.296068  0.001271  0.702535  1.266870e-04  2017-05-12T08:18:3

In [17]:
timelist = []
for index, row in predicted013.iterrows():
    if row.max() < 0.5:
        timelist.append(index)

In [3]:
test = pd.read_csv("./models/20180806/predict/predicted-013013_20170407_000.csv")
test.head()

,NO_LABEL,ROLL,RUN,DOOR
0,0.212231,0.000395,0.785748,0.001626
1,0.233211,0.001141,0.764171,0.001477
2,0.235122,0.001388,0.762440,0.001049
3,0.233136,0.001322,0.764637,0.000905
4,0.232868,0.001290,0.764902,0.000940


## 曖昧データ処理

### timestamp抽出csv変換

In [2]:
import MSclass
import glob

In [ ]:
osmkdir("./models/20180806/")

In [6]:
for A in [0.5,0.6,0.7,0.8,0.9,1]:
    MSclass.predata("./models/20180806/predict/",
                   A,
                   "./models/20180806/class/"
                  )
    

### readcsv for time

In [4]:
import create_model
create_model.read_csv()

TypeError: read_csv() missing 5 required positional arguments: 'accel_file', 'gyro_file', 'label_file', 'timestep', and 'timeshift'

### 再学習

In [ ]:
    import itertools
    directory_list = ["001", "005", "010", "011", "013"]
    combi_list = list(itertools.combinations(directory_list,4))
    outeval = "./models/20180807/act_eval/eval%s_%s_%s_%s.csv" %combi

In [2]:
import pandas as pd
classcsv = pd.read_csv("./models/20180806/class/0.1-001001_20170406_000.csv")
accel = pd.read_csv("./cepstrums/001/001_20170406_000.accel.csv")
gyro = pd.read_csv("./cepstrums/001/001_20170406_000.gyro.csv")
label = pd.read_csv("./cepstrums/001/001_20170406_000.label.csv")

accel[accel["timestamp"] == classcsv["timestamp"]]

ValueError: Series lengths must match to compare

In [2]:
import numpy as np
from keras.datasets import mnist
from keras.models import model_from_json
from keras.utils import np_utils
from keras.optimizers import Adam
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import os.path

import create_model

#pra
batch_size = 100
epochs = 40
units = 180
timestep = 3
timeshift = 1

#ファイルの読み込み

_edata, _elabel = create_model.read_csv()#曖昧データの読み込み
_elabel = np_utils.to_categorical(np.argmax(_elabel[["NO_LABEL", "ROLL", "RUN", "DOOR"]].values, axis = -1), num_classes = 4)

[]
f_model = './models/20180806/model/'
model_filename = []
f_weithts = "./models/20180806/weight/"
weights_filename = []

#曖昧dataの読み込み

#random seed を固定
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
session_conf = tf.ConfigProto(
intra_op_parallelism_threads=1,
inter_op_parallelism_threads=1
)
tf.set_random_seed(1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
KTF.set_session(sess)#固定終わり

with open(outeval, "wt") as f:

    json_string = open(os.path.join(f_model, model_filename)).read()
    model = model_from_json(json_string)    
    model.summary()
    
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.5), metrics=['accuracy'])
    model.load_weights(os.path.join(f_model,weights_filename))

    cbks = []

    history = model.fit(xdata, xlabel, batch_size = 100, epochs = epochs, callbacks = []
                  , validation_data = [ydata, ylabel], class_weight = _class_weight
                  , verbose = 2)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

    #F値の処理
    try:
        _tlabel = np.argmax(_elabel, axis = -1)
        _plabel = np.argmax(predicted, axis = -1)
        for _t, _p in zip(_tlabel, _plabel):
            if _t == _p: counts[_p, 0] += 1
            counts[_p, 1] += 1
            counts[_t, 2] += 1
        _macro_fmesure = 0.0
        _fmesures = []
        for _i in range(4):
            if counts[_i, 1] == 0 or counts[_i, 2] == 0: continue
            _precision = counts[_i, 0] / counts[_i, 1]
            _recall = counts[_i, 0] / counts[_i, 2]
            _fmesure = 2 * (_precision * _recall) / (_precision + _recall)
            _macro_fmesure += _fmesure
            _fmesures.append(_fmesure)
        _macro_fmesure /= 4

        line = [path.basename(_accel), path.basename(_gyro), path.basename(_label), str(_edata.shape[0])
                , "{:0.8f}".format(eval[0]), "{:0.8f}".format(eval[1]), "{:0.8f}".format(_macro_fmesure)]
        line += ["{:0.8f}".format(v) for v in _fmesures]
        f.write(",".join(line))
        f.write("\n")
    except:
        print("ERROR", _edata, predicted)
        _macro_fmesure = 0.0
        line = [path.basename(_accel), path.basename(_gyro), path.basename(_label), str(_edata.shape[0])
                , "{:0.8f}".format(eval[0]), "{:0.8f}".format(eval[1]), "{:0.8f}".format(_macro_fmesure)]
        f.write(",".join(line))
        f.write("\n")
finally:
    session.close()
    print('Goodbye, world!')

SyntaxError: invalid syntax (<ipython-input-2-9d7b43be367b>, line 93)

In [3]:
import MSclass

In [5]:

MSclass.create_vaguedata("./models/20180806/class/",
                        ["011"],
                         "./models/20180809/vague/",
                         0.1
                        )

In [3]:
from relearning import relearning

relearning(f_cep = "./models/20180809/vague/0.1/",
    f_model = "./models/20180806/model/",
    f_weight = "./models/20180806/weight/",
    car_ids =  ["001","005","010","011","013"],
    eval_car_ids = ["001","005","010","011","013"],
    outeval = "./models/20180809/eval/reeval/",
    outmodel = "./models/20180809/model/",
    outweights = "./models/20180809/weight/",
    outpredict = "./models/20180809/predict/",
    batch_size = 100,
    epochs = 40,
    units = 180,
    timestep = 3,
    timeshift = 1,
)

/home/aj1m0n/jupyter_notebook/Datamining/20180731/relearning.py:111: SyntaxWarning: name 'predicted' is assigned to before global declaration
  global predicted #モジュールのグローバル化
Using TensorFlow backend.


['./models/20180809/vague/0.1/001/*.label.csv']
./models/20180806/model/model005_010_011_013.model


/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0
_________________________________________________________________
Trai

TypeError: 'NoneType' object is not subscriptable

In [1]:
from relearning import relearning

relearning(f_cep = "./models/20180809/vague/0.1/",
    f_model = "./models/20180806/model/",
    f_weight = "./models/20180806/weight/",
    car_ids =  ["001","005","010","011","013"],
    eval_car_ids = ["001","005","010","011","013"],
    outeval = "./models/20180809/eval/reeval/",
    outmodel = "./models/20180809/model/",
    outweights = "./models/20180809/weight/",
    outpredict = "./models/20180809/predict/",
    batch_size = 100,
    epochs = 40,
    units = 180,
    timestep = 3,
    timeshift = 1,
)

Using TensorFlow backend.


./models/20180809/vague/0.1/001/001_20170407_000.vague.label.csv
001_20170407_000
['./models/20180809/vague/0.1/001/001_20170407_000.vague.label.csv']
./models/20180806/model/model005_010_011_013.model


/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0
_________________________________________________________________
Trai

KeyboardInterrupt: 

In [3]:
from relearning import relearning

relearning(f_cep = "./models/20180809/vague/0.1/",
    f_model = "./models/20180806/model/",
    f_weight = "./models/20180806/weight/",
    car_ids =  ["001","005","010","011","013"],
    eval_car_ids = ["001","005","010","011","013"],
    outeval = "./models/20180813/eval/reeval/",
    outmodel = "./models/20180813/model/",
    outweights = "./models/20180813/weight/",
    outpredict = "./models/20180813/predict/",
    batch_size = 100,
    epochs = 40,
    units = 180,
    timestep = 3,
    timeshift = 1,
)

./models/20180809/vague/0.1/001/001_20170407_000.vague.label.csv
001_20170407_000
['./models/20180809/vague/0.1/001/001_20170407_000.vague.label.csv']
./models/20180806/model/model005_010_011_013.model


/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0
_________________________________________________________________
Trai

## 3rdlearning

このレイヤーで段階的学習を行う<br>
ex.
1. 車ABCDで学習を行ったモデルを車Aに適応
2. 1日の確率で表せれた予測値からmargin sampleing(max - 2nd max)を行う
3. その差が0.1以下のデータとそれに対応したラベルデータを入力としてモデルを再学習
4. そのモデルで車Eの別日を予測
5. 2に戻る

### create vague data

### 3nd学習

ex.
1. 20170406のサンプリングデータで再学習させたモデルを適用
2. データは 20170406 20170407 2017040511 20170412 (ただし20170406は既にモデルに組まれているので無視をする)
3. 20170407以降のデータを参照前回のreevalからaccracyが上がっているかを確認する
4. 20170407のモデルを適用、20170511以降のデータを見る

In [2]:
import MSclass
for A in [0.1]:
    MSclass.predata("./models/20180813/predict/010_20170406_000/",
                   A,
                   "./models/20180813/class/"
                  )
MSclass.create_vaguedata("./models/20180813/class/",
                        ["013"],
                         "./models/20180813/vague/",
                         0.1
                        )

In [14]:
from relearning import relearning

relearning(f_cep = "./models/20180813/vague/0.1/",
    f_model = "./models/20180810/model/010_20170406_000/",
    f_weight = "./models/20180810/weight/010_20170406_000/",
    car_ids =   ["010"],
    eval_car_ids =  ["001","005","010","011","013"],
    outeval = "./models/20180813/eval/3rdeval/",
    outmodel = "./models/20180813/model/3rdmodel/",
    outweights = "./models/20180813/weight/3rdweight/",
    outpredict = "./models/20180813/predict/3rdpredict/",
    batch_size = 100,
    epochs = 23,
    units = 180,
    timestep = 3,
    timeshift = 1,
)

[['./models/20180810/model/010_20170406_000/re-model001_005_011_013.model']]
['./models/20180813/vague/0.1/010/010_20170512_000.vague.label.csv', './models/20180813/vague/0.1/010/010_20170511_000.vague.label.csv', './models/20180813/vague/0.1/010/010_20170406_000.vague.label.csv', './models/20180813/vague/0.1/010/010_20170407_000.vague.label.csv']
./models/20180813/vague/0.1/010/010_20170512_000.vague.label.csv
010_20170512_000
['./models/20180813/vague/0.1/010/010_20170512_000.vague.label.csv']


/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


./models/20180810/model/010_20170406_000/re-model001_005_011_013.model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0

### accracy Comparison

In [ ]:
#plot

### 4th学習

In [9]:
import MSclass
for A in [0.1]:
    MSclass.predata("./models/20180813/predict/010_20170407_000/",
                   A,
                   "./models/20180813/class/4th/"
                  )
MSclass.create_vaguedata("./models/20180813/class/4th/",
                        ["010","013"],
                         "./models/20180813/vague/4th/",
                         0.1
                        )

In [11]:
from relearning import relearning

relearning(f_cep = "./models/20180813/vague/0.1/",
    f_model = "./models/20180810/model/013_20170407_000/",
    f_weight = "./models/20180810/weight/013_20170407_000/",
    car_ids =  ["010"],
    eval_car_ids = ["010"],
    outeval = "./models/20180813/eval/4theval/",
    outmodel = "./models/20180813/model/4thmodel/",
    outweights = "./models/20180813/weight/4thweight/",
    outpredict = "./models/20180813/predict/4thpredict/",
    batch_size = 100,
    epochs = 40,
    units = 180,
    timestep = 3,
    timeshift = 1,
)

[['./models/20180810/model/013_20170407_000/re-model001_005_010_011.model']]
['./models/20180813/vague/0.1/010/010_20170512_000.vague.label.csv', './models/20180813/vague/0.1/010/010_20170511_000.vague.label.csv', './models/20180813/vague/0.1/010/010_20170406_000.vague.label.csv', './models/20180813/vague/0.1/010/010_20170407_000.vague.label.csv']
./models/20180813/vague/0.1/010/010_20170512_000.vague.label.csv
010_20170512_000
['./models/20180813/vague/0.1/010/010_20170512_000.vague.label.csv']


/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


./models/20180810/model/013_20170407_000/re-model001_005_010_011.model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0

# 検証

### acc

In [3]:
from relearning import relearning

import pandas as pd
import glob

from statistics import mean
car_ids = ["001","005","010","011","013"]

for car in car_ids:
    relearning(f_cep = "./cepstrums/",
        f_model = "./models/20180806/model/",
        f_weight = "./models/20180806/weight/",
        car_ids =  [car],
        eval_car_ids = [car],
        outeval = "./models/20180816/eval/reeval/",
        outmodel = "./models/20180816/model/",
        outweights = "./models/20180816/weight/",
        outpredict = "./models/20180816/predict/",
        batch_size = 50,
        epochs = 10,
        units = 180,
        timestep = 3,
        timeshift = 1,
    )

['./models/20180806/model/model005_010_011_013.model']
['./cepstrums/001/001_20170407_000.label.csv', './cepstrums/001/001_20170406_000.label.csv']
./cepstrums/001/001_20170407_000.label.csv
001_20170407_000
['./cepstrums/001/001_20170407_000.label.csv']


/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


./models/20180806/model/model005_010_011_013.model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0
___________________

### 追加データ量の削減

In [9]:
import pandas as pd
import relearning
from relearning import relearning
    

/home/aj1m0n/jupyter_notebook/Datamining/20180731/relearning.py:125: SyntaxWarning: name 'predicted' is assigned to before global declaration
  global predicted #モジュールのグローバル化
/home/aj1m0n/jupyter_notebook/Datamining/20180731/relearning.py:267: SyntaxWarning: name 'predicted' is assigned to before global declaration
  global predicted #モジュールのグローバル化
Using TensorFlow backend.
/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:178: SyntaxWarning: name 'predicted' is assigned to before global declaration
  global predicted #モジュールのグローバル化


In [20]:
label = pd.read_csv("./cepstrums/001/001_20170406_000.label.csv", index_col=0)
accel = pd.read_csv("./cepstrums/001/001_20170406_000.accel.csv", index_col=0)
gyro = pd.read_csv("./cepstrums/001/001_20170406_000.gyro.csv",index_col=0)

for i in range(int(len(label) / 100)):
        label2 = label.iloc[i*100:len(test) - i * 100]
        #label2 = label2.set_index("timestamp")
        label2.to_csv("./models/20180817/data/001/%s001_20170406_000.label.csv" %i)
        accel2 = accel.iloc[i*100:len(test) - i * 100]
        #accel2 = accel.set_index("timestamp")
        accel2.to_csv("./models/20180817/data/001/%s001_20170406_000.accel.csv" %i)
        gyro2 = gyro.iloc[i*100:len(test) - i * 100]
        #gyro2 = gyro2.set_index("timestamp")
        gyro2.to_csv("./models/20180817/data/001/%s001_20170406_000.gyro.csv" %i)

In [21]:
label.iloc[1:50]

,car_id,NO_LABEL,ROLL,RUN,DOOR
timestamp,,,,,
2017-04-06T07:26:34.338,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:35.338,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:36.338,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:37.338,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:38.338,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:39.340,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:40.338,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:41.338,1,0.0,0.0,0.0,0.000000
2017-04-06T07:26:42.338,1,0.0,0.0,0.0,0.000000


In [23]:
relearning(f_cep = "./models/20180817/data/",
                    f_model = "./models/20180806/model/",
                    f_weight = "./models/20180806/weight/",
                    car_ids =  ["001"],
                    eval_car_ids = ["001"],
                    outeval = "./models/20180816/eval/reeval/",
                    outmodel = "./models/20180816/model/",
                    outweights = "./models/20180816/weight/",
                    outpredict = "./models/20180816/predict/",
                    batch_size = 50,
                    epochs = 10,
                    units = 180,
                    timestep = 3,
                    timeshift = 1,
                )

['./models/20180806/model/model005_010_011_013.model']
['./models/20180817/data/001/15001_20170406_000.label.csv', './models/20180817/data/001/7001_20170406_000.label.csv', './models/20180817/data/001/1001_20170406_000.label.csv', './models/20180817/data/001/34001_20170406_000.label.csv', './models/20180817/data/001/6001_20170406_000.label.csv', './models/20180817/data/001/21001_20170406_000.label.csv', './models/20180817/data/001/14001_20170406_000.label.csv', './models/20180817/data/001/5001_20170406_000.label.csv', './models/20180817/data/001/0001_20170406_000.label.csv', './models/20180817/data/001/38001_20170406_000.label.csv', './models/20180817/data/001/31001_20170406_000.label.csv', './models/20180817/data/001/10001_20170406_000.label.csv', './models/20180817/data/001/16001_20170406_000.label.csv', './models/20180817/data/001/28001_20170406_000.label.csv', './models/20180817/data/001/20001_20170406_000.label.csv', './models/20180817/data/001/25001_20170406_000.label.csv', './mo

/home/aj1m0n/jupyter_notebook/Datamining/20180731/create_model.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  _labels["date"] = np.datetime_as_string(labels.timestamp, "D")#timeごとに整列化


./models/20180806/model/model005_010_011_013.model
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           (None, 3, 180)            0         
_________________________________________________________________
LSTM (LSTM)                  (None, 180)               259920    
_________________________________________________________________
Dense1 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense2 (Dense)               (None, 180)               32580     
_________________________________________________________________
Dense3 (Dense)               (None, 4)                 724       
_________________________________________________________________
out (Dense)                  (None, 4)                 20        
Total params: 325,824
Trainable params: 325,824
Non-trainable params: 0
___________________

### plot

In [25]:
import glob
import pandas as pd

In [34]:
test = glob.glob("./models/20180817/eval/*/001eval005_010_011_013.csv")
ALLdata = pd.DataFrame(columns=["range","accel","gyro","label","data_counts","loss","accuracy","macro-fmesure","NO_LABEL","ROLL","RUN","DOOR"])
for i in test:
    df = pd.read_csv(i)
    _df = df[df["accel"] == "001_20170407_000.accel.csv"]
    _df["range"] = i.replace("./models/20180817/eval/","").replace("001_20170406_000/001eval005_010_011_013.csv","")
    ALLdata = pd.concat([ALLdata,_df])
ALLdata.to_csv("./models/20180817/eval/ALLdata.csv")

/home/aj1m0n/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
ALLdata.sort_values(by=["range"], ascending=True)

['./models/20180817/eval/14001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/24001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/2001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/26001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/38001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/11001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/15001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/17001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/22001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/3001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/37001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/5001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/8001_20170406_000/001eval005_010_011_013.csv',
 './models/20180817/eval/23001_20170406_000/001eval005_

In [36]:
(len(label)- 29*100*2)/len(label)

0.25621954347268533

In [37]:
(len(label)- 34*100*2)/len(label)

0.12798153372659657

In [39]:
len(label)- 38*100*2

198

### margin sampling

In [7]:
import pandas as pd
import MSclass
from statistics import mean

In [2]:
MSclass.marginsampling("./models/20180809/predict/", ["001"], "./models/20180817/predict/")

In [33]:
max001 = pd.read_csv("./models/20180817/predict/001/001001_20170406_000.csv.csv")
window = len(pd.read_csv("./models/20180817/data/001/38001_20170406_000.label.csv"))
while window <= len(max001):
    for slid in range(len(max001)+1-window):
        if slid == 0:
            minest = max001.iloc[slid:slid + window]
        elif mean(minest["margin"]) > mean(max001.iloc[slid:slid + window]["margin"]):
            minest = max001.iloc[slid:slid + window]
    minest.to_csv("./models/20180817/data/%s.csv"%window)
    window += 200

In [27]:
mean(max001.iloc[slid:slid +window]["margin"])

0.9497981684885977

In [30]:
mean(minest["margin"])

StatisticsError: mean requires at least one data point

In [31]:
minest

,timestamp,margin
